## merge(类似MySQL内连接)

In [10]:
import pandas as pd

'''
Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will
be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on.

how : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’
    Type of merge to be performed.
    left: use only keys from left frame, similar to a SQL left outer join; preserve key order.
    right: use only keys from right frame, similar to a SQL right outer join; preserve key order.
    outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
    inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

on : label or list
    Column or index level names to join on. These must be found in both DataFrames. If on is None and
    not merging on indexes then this defaults to the intersection of the columns in both DataFrames.

left_on : label or list, or array-like
    Column or index level names to join on in the left DataFrame. Can also be an array or list of arrays
    of the length of the left DataFrame. These arrays are treated as if they are columns.

right_on : label or list, or array-like
    Column or index level names to join on in the right DataFrame. Can also be an array or list of arrays of the
    length of the right DataFrame. These arrays are treated as if they are columns.

left_index : bool, default False
    Use the index from the left DataFrame as the join key(s). If it is a MultiIndex, the number of keys in the other
    DataFrame (either the index or a number of columns) must match the number of levels.

right_index : bool, default False
    Use the index from the right DataFrame as the join key. Same caveats as left_index.

suffixes : tuple of (str, str), default (‘_x’, ‘_y’)
    Suffix to apply to overlapping column names in the left and right side, respectively. To raise an
    exception on overlapping columns use (False, False).
'''
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': [3, 2, 1, 7, 4, 5]})

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
print(df1, '\n\n', df2)

  key  data1
0   b      3
1   b      2
2   a      1
3   c      7
4   a      4
5   b      5 

   key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [11]:
# 默认how='inner';两张表都有的列或索引的交集作为连接键
df1.merge(df2, on='key')

,key,data1,data2
0,b,3,1
1,b,3,3
2,b,2,1
3,b,2,3
4,b,5,1
5,b,5,3
6,a,1,0
7,a,1,2
8,a,4,0
9,a,4,2


In [12]:
df1  # df1不变

,key,data1
0,b,3
1,b,2
2,a,1
3,c,7
4,a,4
5,b,5


In [13]:
# 两张表都有的列或索引的并集作为连接键
df1.merge(df2, on='key', how='outer')

,key,data1,data2
0,b,3.0,1.0
1,b,3.0,3.0
2,b,2.0,1.0
3,b,2.0,3.0
4,b,5.0,1.0
5,b,5.0,3.0
6,a,1.0,0.0
7,a,1.0,2.0
8,a,4.0,0.0
9,a,4.0,2.0


In [14]:
# 左表的列或索引作为连接键
df1.merge(df2, on='key', how='left')

,key,data1,data2
0,b,3,1.0
1,b,3,3.0
2,b,2,1.0
3,b,2,3.0
4,a,1,0.0
5,a,1,2.0
6,c,7,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [15]:
# 指定右表的列或索引作为连接键
df1.merge(df2, on='key', how='right')

,key,data1,data2
0,a,1.0,0
1,a,4.0,0
2,b,3.0,1
3,b,2.0,1
4,b,5.0,1
5,a,1.0,2
6,a,4.0,2
7,b,3.0,3
8,b,2.0,3
9,b,5.0,3


In [16]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df3, '\n\n', df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6 

   rkey  data2
0    a      0
1    b      1
2    d      2


In [17]:
# left_on和right_on指定的列不会合并成一列
df3.merge(df4, left_on='lkey', right_on='rkey')  # 指定左右表用作连接键的列

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [18]:
df11 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                     'data1': [3, 2, 1, 7, 4, 5]})
df12 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                     'data1': [3, 2, 1, 7, 4, 5]})

# 默认suffixes=["_x", "_y"]
df11.merge(df12, on='key')

,key,data1_x,data1_y
0,b,3,3
1,b,3,2
2,b,3,5
3,b,2,3
4,b,2,2
5,b,2,5
6,b,5,3
7,b,5,2
8,b,5,5
9,a,1,1


In [19]:
# 指定left DataFrame重名列的后缀为_left
# 指定right DataFrame重名列的后缀为_right
df11.merge(df12, on='key', suffixes=["_left", "_right"])

,key,data1_left,data1_right
0,b,3,3
1,b,3,2
2,b,3,5
3,b,2,3
4,b,2,2
5,b,2,5
6,b,5,3
7,b,5,2
8,b,5,5
9,a,1,1
